# Loschmidt Analysis Walkthrough

This notebook walks through the analysis routines available for the `recirq.otoc.loschmidt.tilted_sqare_lattice` analysis routines. In particular, you will be guided on how to group, slice, fit, and plot data to extract fidelities from loschmidt echo data.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Set up reasonable defaults for figure fonts
import matplotlib
matplotlib.rcParams.update(**{
    'axes.titlesize': 14,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'legend.title_fontsize': 12,
    'figure.figsize': (7, 5),
})

## Load Raw Results

This notebook assumes you have executed the `run-simulator.py` script in this directory to execute the algorithmic benchmark on a simulator, and that the results were saved with run_id `"simulated-1"`. Otherwise, please modify the `run_id` parameter below.

After some standard imports, we load the `ExecutableGroupResult` in its entirety using `cirq_google.workflow` tools. This is the complete raw dataset.

In [ ]:
import pandas as pd
import numpy as np

import cirq_google as cg

import recirq.otoc.loschmidt.tilted_square_lattice.analysis as analysis

In [ ]:
raw_results = cg.ExecutableGroupResultFilesystemRecord.from_json(run_id='simulated-1').load()
repr(raw_results)[:100] + ' ...'

## Use Pandas for slicing and dicing

We can extract the most relevant parameters from the raw results and flatten it into a tabular format. In particular, we use `pd.DataFrame` for further data aggregation and plotting.

The measured `success_probability` ranges between 0 and 1, indicating the fraction of times the measured result after the echo actually returned to its initial state.

In [ ]:
df = analysis.loschmidt_results_to_dataframe(raw_results)
print(len(df), 'rows')
df.head()

## Aggregation

Instead of considering each result in isolation, we can aggregate quantities to make more meaningful plots and fits.

In the following, we use pandas group-by functionality to
 1. Average (and compute the standard deviation) over random instances, holding all else constant.
 2. Plot these averaged quantities vs. macrocycle_depth, holding all else constant.
 
We use `analysis.groupby_all_except` as a wrapper around `pd.DataFrame.groupby` so we can specify what we _don't_ want to aggregate over; which should make these analysis routines more extensible.

### (1) Mean and std

In [ ]:
agg_df1, gb_cols1 = analysis.groupby_all_except(
    df.drop(['n_qubits', 'q_area'], axis=1), 
    y_cols=('instance_i', 'success_probability'), 
    agg_func={'success_probability': ['mean', 'std']}
)
agg_df1

### (2) vs. macrocycle_depth

In [ ]:
agg_df2, gb_cols2 = analysis.groupby_all_except(
    agg_df1, 
    y_cols=('macrocycle_depth', 'success_probability_mean', 'success_probability_std'),
    agg_func=list
)
agg_df2

In [ ]:
for i, row in agg_df2.iterrows():
    label = ' '.join(f'{cc}={row[cc]}' for cc in gb_cols2)
    plt.errorbar(
        x=row['macrocycle_depth'],
        y=row['success_probability_mean'],
        yerr=row['success_probability_std'],
        label=', '.join(f'{row[col]}' for col in gb_cols2),
        capsize=5, ls='', marker='o',
    )
    
plt.xlabel('Macrocycle Depth')
plt.ylabel('Success Probability')
plt.legend(title=','.join(gb_cols2), loc='best')
plt.tight_layout()

## Fitting

Part of the loschmidt echo analysis involves fitting an exponential decay in success probability vs. macrocycle_depth to robustly estimate a per-macrocycle error rate.

### Reshape data for fitting

Now, we group all `(macrocycle_depth, instance_i)` points into a list holding everything else constant so we can use the raw points for fitting. We'll get 3 fits (one for each row) with the example dataframe in this notebook. The following cell shows that groupby operation used under the hood in `analysis.fit_vs_macrocycle_depth`.

In [ ]:
agg_df3, gb_cols3 = analysis.groupby_all_except(
    df.drop('n_qubits', axis=1),
    y_cols=('instance_i', 'macrocycle_depth', 'q_area', 'success_probability'),
    agg_func=list,
)
agg_df3

In [ ]:
fit_df, exp_ansatz = analysis.fit_vs_macrocycle_depth(df)
fit_df

## Merging and Plotting

To plot the mean+stddev data as well as visualizations of the fits, we merge (join) the two dataframes. Note that the groupby columns of the two dataframes are the same, so we can join on them:

In [ ]:
print(gb_cols2)
print(gb_cols3)

In [ ]:
total_df = pd.merge(agg_df2, fit_df, on=gb_cols2)
total_df

In [ ]:
colors = plt.get_cmap('tab10')

for i, row in total_df.iterrows():
    plt.errorbar(
        x=row['macrocycle_depth'],
        y=row['success_probability_mean'],
        yerr=row['success_probability_std'],
        marker='o', capsize=5, ls='',
        color=colors(i),
        label=f'{row["width"]}x{row["height"]} ({row["n_qubits"]}q) {row["processor_id"]}; f={row["f"]:.3f}'
    )
    
    xx = np.linspace(np.min(row['macrocycle_depth']), np.max(row['macrocycle_depth']))
    yy = exp_ansatz(xx, a=row['a'], f=row['f'])
    plt.plot(xx, yy, ls='--', color=colors(i))
    
plt.legend(loc='best')
plt.yscale('log')
plt.xlabel('Macrocycle Depth')
plt.ylabel('Success Probability')
plt.tight_layout()

## Fit vs. "Quantum Area"

In a local depolarizing model, we expect success to decay exponentially in circuit depth and the number of qubits. We define a quantity called quantum area (`q_area`) which is the circuit width (i.e. number of qubits) multiplied by its depth. This is the number of operations in the circuit (also including any idle operations).

By defining this new quantity, we can fit a curve of fidelity vs. quantum area.. The following cell shows the groupby operation used in `analysis.fit_vs_q_area`.

In [ ]:
agg_df4, gb_cols4 = analysis.groupby_all_except(
    df.drop(['width', 'height'], axis=1),
    y_cols=('q_area', 'n_qubits', 'instance_i', 'macrocycle_depth', 'success_probability'),
    agg_func=list,
)
agg_df4

In [ ]:
fit_df2, exp_ansatz_vs_q_area = analysis.fit_vs_q_area(df)
fit_df2

Once again, we'll merge (join) the raw data with the fits data. This seems like overkill since everthing has been grouped into one DataFrame row, but this code will run without modification when comparing multiple runs or multiple processors.

In [ ]:
total_df2 = pd.merge(agg_df4, fit_df2, on=gb_cols4)
total_df2

In [ ]:
colors = plt.get_cmap('tab10')

for i, row in total_df2.iterrows():
    plt.scatter(row['q_area'], row['success_probability'], color=colors(i))
    
    xx = np.linspace(np.min(row['q_area']), np.max(row['q_area']))
    yy = exp_ansatz_vs_q_area(xx, a=row['a'], f=row['f'])
    plt.plot(xx, yy, ls='--', color=colors(i),
             label=f'{row["run_id"]}; f={row["f"]:.3f}'
            )


plt.legend(loc='best')
plt.xlabel('Quantum Area')
plt.ylabel('Macrocycle Fidelity')
plt.yscale('log')
plt.tight_layout()